# Prerequisites

- RSA
- Coppersmith attacks

# Theory

Explanation https://youtu.be/RgbrpmJ49r4?t=2321

**Theorem** Howgrave-Graham  
Given a polynomial $f$ of degree d and integer N, we can find in polynomial time roots $r$ modulo divisors $B$ of $N$ satisfying
$$f(r) \equiv 0 \ mod \ B$$ for $|B| > N^\beta$ when $|r| < N^{\beta^2 / d}$

Same idea as Coppersmith but we find a vector in a lattice less than $N^\beta < B$

**Theorem**:
    
Given half of the MSB / LSB bits of a factor p, we can factor and RSA modulus N = pq in polynomial time

**Proof**:

Let $f(x) = x + a$ where $a$ are the MSB of $p$, and $r$ are theLSB of $p$, so $a+r = p$
- $f(x) \equiv 0 \ mod \ p > N^{1/2}$
- Apply Howgrave-Graham for $d = 1$ and $\beta = 1/2 => |r| < N^{\beta^2/d} = N^{1/4}$


We chose the polynomial basis:
$(x+a)^2, x+a, N$ which corresponds to the lattice
$\begin{bmatrix}
R^2 & 2Ra & a^2 \\
0 &R &a \\
0 &0 &N
\end{bmatrix}
$
With $\dim L = 3, \det L = R^3N$

From LLL => $|v| \approx \det L^{1/\dim L}$

The algorithm will find the root $r$ when $|g(r)| \leq |v| \approx \det L^{1/\dim L} <p $

# Code

## Toy implementation

In [1]:
p = random_prime(2^512)
q = random_prime(2^512)
N = p*q

In [3]:
#Delete 86 bits
a = p - (p%2^86)
hex(a)

'0x67edf50300cb19742236e645ccf36e5c723a1028b122942de22146d657538417e0dc0146836ea43c415fadd7a763c00efd9264e4bcc000000000000000000000'

In [4]:
#Define the bound
R = 2^86
M = matrix([
    [R^2, 2*R*a, a^2],
    [0, R, a],
    [0, 0, N]
])

In [6]:
B = M.LLL()

In [7]:
Q = sum([B[0][i]* x^(2-i) / R^(2-i) for i in range(3)])

In [10]:
a + Q.roots(ring = ZZ)[0][0] == p

True

## Sage implementation

Sage has a function small_roots that applies coppersmith's theorem

Source for code: https://github.com/ashutosh1206/Crypton/tree/master/RSA-encryption/Attack-Coppersmith

In [1]:
p = random_prime(2^512)
q = random_prime(2^512)
N = p*q
#Delete 86 bits
a = p - (p%2^86)
hex(a)

'0xabfe79e046d21865083e5370b6abb6f4430b09d9a381b5e623c2a86f187ff173b4699eed095ebb49ddcbf989cf42bbe688148601140000000000000000000000'

In [2]:
f = x + a

In [4]:
def factorization_n(f, N):
    P.<x> = PolynomialRing(Zmod(N))
    f = P(f)
    beta = .5
    dd = f.degree() # Degree of the polynomial (e for the above polynomial)
    epsilon = beta/7 #
    R = ceil(N**((beta**2/dd) - epsilon)) #The bound
    r = f.small_roots(R, beta, epsilon)
    return r

In [7]:
r = factorization_n(f, N)

In [10]:
a + r[0] == p

True

# Resources:

- Coppersmith paper: https://cr.yp.to/bib/2001/coppersmith.pdf
- https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.86.9408&rep=rep1&type=pdf